In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

In [2]:
# %load /Users/janclaes/GitHub/Instacart/SCRIPTS/ReadIn
import os
import glob
import pandas as pd
from collections import Counter
from itertools import chain

class ReadIn(object):
    '''Read In multiple same file extensions using PANDAS read_ functions'''

    def __init__(self, pwd):
        '''set directory from which to import'''
        self.pwd = pwd

    def GetCSV(self):
        self.all_files = glob.glob(os.path.join(self.pwd, self.ext))

    def MakeDICT(self,**kwargs):
        ''' ext= '*.csv'
            ext= '*.xlsx'
                
            for sheets : sheets = '<name>'
        '''
        self.ext = kwargs.get('ext')
        self.sheets = kwargs.get('sheets')
        self.GetCSV()
        self.datasets = {}
        for key in self.all_files:
            file_name = key.split('/')[-1].strip(self.ext)
            self.datasets.update({file_name: key})

        self.ReadCSVs()

    def ReadCSVs(self):
        file_extensions = {'*.csv':"pd.read_csv", '*.xlsx':"pd.read_excel", '*.xls':"pd.read_excel", '*.json':"pd.read_json"}
        command = file_extensions.get(self.ext)
        columns_list = []
        columns_dict = {}
        
        ## TODO
        #
        # find all excel sheets and import every single one
        
        for key in self.datasets.keys():
            print("Reading in : {}".format(key))
            globals()['%s' %key] = eval(command)(self.datasets.get(key)) #,**kwargs)
            column_list = (eval(key).columns.tolist())
            columns_list.append(column_list)
            columns_dict.update({key:column_list})
            NAcheck = eval(key).isna().any(axis=0).any()
            if NAcheck == True:
                print('  ...\033[1m[{}] contains NAs\033[0m...'.format(key))
        
        print('\nStoring names in [datasets]')
        globals()['datasets'] = list(self.datasets.keys())
        
        print('Storing column names count in [columns_list]')
        globals()['columns_list'] = Counter([i for i in chain.from_iterable([*columns_dict.values()])])
        
        
        print('Storing column names as dictionary in {columns_dict}')
        globals()['columns_dict'] = columns_dict
        
        self.MakeVC()
        
    def MakeVC(self):
        RowNames = [*columns_dict.keys()]
        ColNames = [*columns_list.keys()]

        VC = pd.DataFrame(np.zeros((len(RowNames),len(ColNames))))        
        VC.columns = columns_list
        VC.index = RowNames

        for idx in columns_dict.keys():
            for col in columns_dict[idx]:
                VC.loc[idx,col] = float(len(eval(idx).loc[:,col].unique()))

        print('\nStoring summary table in [VC]\n\n')
        globals()['VC'] = VC.T.replace(0,'') 
        
        for data in datasets:
            print('\033[1m{} \n\033[0m>>> rows: {:,}\n<<< columns: {:,}\n'.format(data, eval(data).shape[0], eval(data).shape[1]))

In [3]:
ReadIn('/Users/janclaes/GitHub/Instacart/instacart-basket-prediction/data/raw/').MakeDICT(ext='*.csv')


Storing names in [datasets]
Storing column names count in [columns_list]
Storing column names as dictionary in {columns_dict}

Storing summary table in [VC]




In [4]:
VC

""


In [5]:
order = order.sort_values(['user_id','order_number'], ascending=True)

NameError: name 'order' is not defined

In [6]:
order.tail(30)

NameError: name 'order' is not defined

In [7]:
def f(user_id,n=2,asc=True):
    globals()['ss'] = str(tuple(order.query('user_id == %s' % user_id).order_id)[:n])
    print(ss)
    out = order_products__prior.query('order_id in %s' % ss)
    out.order_id = out.order_id.astype("category")
    out.order_id = out.order_id.astype("category").cat.set_categories(eval(ss))    
    out = out.sort_values(['order_id'], ascending=asc)
    return out

def ff(n=2):
    priors = order_products__prior.query('order_id == %i' % eval(ss)[-n])\
        .query('reordered > 0')['product_id'].tolist()
    print('at {} reordered {}'.format(eval(ss)[-n], priors))
    
    print('at {}:'.format(eval(ss)[-n-1]))
    return order_products__prior.query('order_id == %i' % eval(ss)[-n-1])\
            .query('product_id in %s' % priors)

In [140]:
df = f(user_id=1468,n=20)

(3144300, 260547, 1905019, 3014999, 2177388, 634358, 36883, 2724741, 98665, 304803)


/Users/janclaes/anaconda/lib/python3.6/site-packages/pandas/core/generic.py:3643: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[name] = value


In [141]:
df.product_id.value_counts()/df.order_id.unique().shape[0]

17211    0.444444
12013    0.444444
13166    0.333333
12817    0.333333
13176    0.333333
35213    0.222222
16234    0.222222
29020    0.222222
204      0.222222
38365    0.111111
42887    0.111111
37646    0.111111
32655    0.111111
46382    0.111111
33153    0.111111
19540    0.111111
18135    0.111111
30233    0.111111
21616    0.111111
48933    0.111111
4258     0.111111
49191    0.111111
46654    0.111111
20711    0.111111
35752    0.111111
31081    0.111111
17835    0.111111
32684    0.111111
16349    0.111111
10673    0.111111
5876     0.111111
1463     0.111111
19068    0.111111
31808    0.111111
Name: product_id, dtype: float64

In [152]:
x,y = 0.11,0.44
xORy = [(x+y),max(x,y)]

In [153]:
xANDy = [x+y-i for i in xORy]

In [154]:
[i/y for i in xANDy]

[0.0, 0.2500000000000001]

In [151]:
df.query('order_id in %s' % df.query('product_id == 17211').order_id.tolist())\
    .product_id.value_counts()/df.query('product_id == 17211').order_id.shape[0]

17211    1.00
12013    1.00
29020    0.50
30233    0.25
1463     0.25
19540    0.25
16234    0.25
31081    0.25
35752    0.25
35213    0.25
33153    0.25
Name: product_id, dtype: float64

In [9]:
f(user_id=3,n=20)
ff()

(1374495, 444309, 3002854, 2037211, 2710558, 1972919, 1839752, 3225766, 3160850, 676467, 521107, 1402502, 2774568)


/Users/janclaes/anaconda/lib/python3.6/site-packages/pandas/core/generic.py:3643: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[name] = value


at 1402502 reordered [39190, 18599, 23650, 21903, 47766, 24810]
at 521107:


,order_id,product_id,add_to_cart_order,reordered
4937733,521107,39190,1,1
4937734,521107,47766,2,1
4937735,521107,21903,3,1


In [10]:
order_products__prior.query('order_id == %i' % eval(ss)[-3])

,order_id,product_id,add_to_cart_order,reordered
4937733,521107,39190,1,1
4937734,521107,47766,2,1
4937735,521107,21903,3,1
4937736,521107,43961,4,1
4937737,521107,17668,5,1


reordered is always zero the first order

In [11]:
ss = str(tuple(order[order.days_since_prior_order.isna()].order_id))
order_products__prior.query('order_id in %s' % ss).reordered.describe()

count    2078068.0
mean           0.0
std            0.0
min            0.0
25%            0.0
50%            0.0
75%            0.0
max            0.0
Name: reordered, dtype: float64